In [ ]:
import pandas as pd
import lithops
from dataretrieval import wqp
import dask.dataframe

In [ ]:
site_list = pd.read_csv('NWQN_sites.csv', comment='#', dtype={'SITE_QW_ID': str, 'SITE_FLOW_ID': str})
site_list

In [ ]:
sites = site_list['SITE_QW_ID'].to_list()[0:8]

In [ ]:
def map_references(site):
    """ Map function to open virtual datasets.
    """
    nawqa, _ = wqp.get_results(siteid=f'USGS-{site}', 
                               # characteristicName='Phosphorus',
                               project="National Water Quality Assessment Program (NAWQA)")

    # Need this in case we get no results from our request
    if len(nawqa) != 0: 
        nawqa.astype(str).to_parquet('nawqa.parquet',
                                     engine='pyarrow',
                                     partition_cols=['MonitoringLocationIdentifier'],
                                     compression='zstd')
        
        return nawqa


# def reduce_references(results):
#     """ Reduce to concat virtual datasets.

#     """
#     combined_vds = xr.combine_nested(
#         results,
#         concat_dim=["Time"],
#         coords="minimal",
#         compat="override",
#     )
#     return combined_vds


fexec = lithops.LocalhostExecutor()

# futures = fexec.map_reduce(
#     map_references,
#     file_pattern,
#     reduce_references,
#     spawn_reducer=100,
# )

futures = fexec.map(
    map_references,
    sites,
)

# futures
ds = futures.get_result()

In [ ]:
x = pd.read_parquet('nawqa.parquet', engine='pyarrow')
x['MonitoringLocationIdentifier'].unique()

In [ ]:
x = dask.dataframe.read_parquet('nawqa.parquet', engine='pyarrow')
x['MonitoringLocationIdentifier'].compute().unique()